# Bayes Logistic Regression by Edward

## Logistic Regression

Setting the input and output as following
\begin{align}
\mathcal{D} = \{(x_1,y_1),\cdots,(x_n,y_1)\}
\end{align}

Logistic Regression Model is defined as following:

\begin{align}
p &= \sigma(wx + b) \\
y &\sim {\rm Bern}(p)
\end{align}

This means that output $y$ is generated by input $x$ based on sigmoid function and Bernoulli distribution ${\rm Bern}(p)$.

Lets Start Model Setting By Edward:

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

tfd = tfp.distributions
tfe = tf.contrib.eager
ed = tfp.edward2

tf.enable_eager_execution()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
#Model Function
def logistic_regression(X):
    w = ed.Normal(loc = tf.zeros(X.shape[1]), scale = 1, name = "coeffs")
    b = ed.Normal(loc = 0., scale = 1, name = "intercept")
    y = ed.Bernoulli(logits = tf.tensordot(X, w, [[1],[0]]) + b, name = "outcomes")
    return y

In Normal class in edward, each parameters are correspond to  

 loc = $\mu$

scale = $\sigma$.

Here, we set $w$ and $b$ by standard normal random variables.

$w$ is model parameter and here, we represent it as probability distribution.

This means that the above logistic regression model is $\bf{Bayes\ model}$.

In [0]:
#Data set and Model running
num_features = 10
X = tf.random_normal([100,num_features])
y = logistic_regression(X)

## See Posterior Distribution

In Bayes analysis, what we'd like to know is that $\bf{posterior\ distribution}$ of model parameter:

$$
p(w, b \mid \mathcal{D}).
$$

In [0]:
def logistic_regression_posterior(num_features):
    posterior_w = ed.MultivariateNormalTriL(
        loc = tf.get_varialbe("w_loc", [num_featues]),
        scale_tril = tfp.trainable_distributions.tril_with_diag_softplus_and_shift(
            tf.get_variable("w_scale",[num_features * (num_features + 1) / 2])),
        name = "w_posterior")
    posterior_b = ed.Normal(
        loc = tf.get_varialbe("b_loc",[]),
        scale = tfp.trainable_distributions.tril_with_diag_softplus_and_shift(
            tf.get_variable("b_scale",[])),
        name = "b_posterior")
    return posterior_w, posterior_b

MultivariateNormalTriL:

    Multivariate Normal distribution Class in Edward.

tf.get_variable(variable name, variable size):

    If variable was defined by tensorflow before code, get the variable.

 tfp.trainable_distributions.tril_with_diga_softplus_and_shift(variable):
 
    change vectors to lower-triangular scale matrices.
    


## Set Prior Distribution

In [0]:
def set_prior_to_posterior_mean(f, *args, **kwargs):
    """Forms posterior predictions, setting each prior to its posterior mean."""
    name = kwargs.get("name")
    if name == "w":
        return posterior_w.distribution.mean()
    elif name == "b":
        return posterior_b.distribution.mean()
    return f(*args, **kwargs)

with ed.interception(set_prior_to_posterior_mean):
    predictions = logistic_regression(X)

## Log Likehood Function

In [0]:
#data
X = tf.random_normal([100,55])
y = tf.random_uniform([100], minval = 0, maxval = 2, dtype = tf.int32)

#Log Likehood Finction
log_joint = ed.make_log_joint_fn(logistic_regression)

In [0]:
def target_log_prob_fn(w,b):
    return log_joint(X, w=w, b=b, y=y)

## MCMC Processing

In [0]:
hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=target_log_prob_fn,
    step_size=0.1,
    num_leapfrog_steps=5)

In [28]:
states, kernel_results = tfp.mcmc.sample_chain(
    num_results=1000,
    current_state=[tf.random_normal([55]), tf.random_normal([])],
    kernel=hmc_kernel,
    num_burnin_steps=500)

LookupError: ignored

***Not Worked Well***